In [23]:
import pandas as pd
import os


In [42]:
dfs = [pd.read_parquet(f"./DNS-Tunnel-Datasets/normal/normal/{f}") for f in 
              os.listdir('./DNS-Tunnel-Datasets/normal/normal/') if f.endswith('.pq')]

cols_unwanted = ['payload.ack', 'payload.dataofs', 'payload.flags', 'payload.reserved', 'payload.seq', 'payload.urgptr', 'payload.window']
dfs = [df[[c for c in df.columns if c not in cols_unwanted]] for df in dfs]

df_normal = pd.concat(dfs)

dfs = [pd.read_parquet(f"./DNS-Tunnel-Datasets/tunnel/{f}") for f in 
              os.listdir('./DNS-Tunnel-Datasets/tunnel/') if f.endswith('.pq')]

cols_unwanted = ['payload.ack', 'payload.dataofs', 'payload.flags', 'payload.reserved', 'payload.seq', 'payload.urgptr', 'payload.window']
dfs = [df[[c for c in df.columns if c not in cols_unwanted]] for df in dfs]

df_tunneling = pd.concat(dfs)
df_normal['label'] = 0
df_tunneling['label'] = 1

df_all = pd.concat([df_normal, df_tunneling]).copy()


/var/folders/vt/4l69jtfd3m5blk3vs5plrg_40000gp/T/ipykernel_90415/1072533817.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_normal = pd.concat(dfs)
/var/folders/vt/4l69jtfd3m5blk3vs5plrg_40000gp/T/ipykernel_90415/1072533817.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([df_normal, df_tunneling]).copy()


In [43]:
df_all.dtypes

version                      int64
ihl                        float64
tos                        float64
len                        float64
id                         float64
flags                      float64
frag                       float64
ttl                        float64
proto                      float64
chksum                     float64
src                         object
dst                         object
payload.sport                int64
payload.dport                int64
payload.len                float64
payload.chksum               int64
payload.payload.length     float64
payload.payload.id           int64
payload.payload.qr           int64
payload.payload.opcode       int64
payload.payload.aa           int64
payload.payload.tc           int64
payload.payload.rd           int64
payload.payload.ra           int64
payload.payload.z            int64
payload.payload.ad           int64
payload.payload.cd           int64
payload.payload.rcode        int64
payload.payload.qdco

In [44]:
df_all.drop('src', axis=1, inplace=True)
df_all.drop('dst', axis=1, inplace=True)

df_all.drop('payload.payload.ns', axis=1, inplace=True)
df_all.drop('payload.payload.ar', axis=1, inplace=True)
          

In [45]:
df_all['qname_first_part'] = df_all['qname'].apply(lambda x:x.split('.')[0])
df_all['qname_tld_part'] = df_all['qname'].apply(lambda x:x.split('.')[-2])

In [48]:
# sort columns alphabetically
df_all.sort_index(axis=1, inplace=True)

In [56]:
for i, v in enumerate(df_all.dtypes):
    print(i, v)
    

0 float64
1 float64
2 float64
3 float64
4 float64
5 float64
6 float64
7 int64
8 float64
9 float64
10 int64
11 int64
12 float64
13 int64
14 int64
15 int64
16 int64
17 int64
18 int64
19 float64
20 int64
21 int64
22 int64
23 int64
24 int64
25 int64
26 int64
27 int64
28 int64
29 int64
30 float64
31 float64
32 int64
33 object
34 object
35 object
36 int64
37 float64
38 float64
39 float64
40 int64
41 int64


In [ ]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn import metrics

# 1. Prepare Your Data

X = df_all[[c for c in df_all.columns if c!='label']]
y = df_all['label']

# Identify categorical features (CatBoost handles them directly)
categorical_features_indices = [32, 33, 34, 35]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 2. Create CatBoost Data Pool (Recommended for Efficiency)
train_pool = Pool(
    data=X_train,
    label=y_train,
    cat_features=categorical_features_indices
)

test_pool = Pool(
    data=X_test,
    label=y_test,
    cat_features=categorical_features_indices
)

# 3. Initialize and Train the CatBoost Model
# You can customize hyperparameters here
model = CatBoostClassifier(
    iterations=100,         # Number of boosting iterations (trees)
    learning_rate=0.1,      # Step size shrinkage to prevent overfitting
    depth=2,                # Depth of the trees
    loss_function='Logloss', # For binary classification
    eval_metric='Accuracy',  # Metric to monitor during training
    random_seed=42,
    verbose=0               # Set to 0 to suppress verbose output, 1 for progress
)

model.fit(train_pool, eval_set=test_pool, early_stopping_rounds=10)

# 4. Make Predictions
y_pred_proba = model.predict_proba(test_pool)
y_pred = model.predict(test_pool)


# 5. Evaluate the Model
accuracy = metrics.accuracy_score(y_test, y_pred)
print(f"Accuracy on the test set: {accuracy:.4f}")


accuracy = metrics.f1_score(y_test, y_pred)
print(f"F1 score on the test set: {accuracy:.4f}")

# 6. (Optional) Get Feature Importances
feature_importances = model.get_feature_importance()
feature_names = X.columns
for importance, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print(f"Feature: {name}, Importance: {importance:.4f}")
